In [1]:
import sys
sys.path.append('../')
from matformer.tensors_dataclasses import TensorDC, NormalTensor, PaddedTensor, UnpaddedTensor
import torch
from matformer.matformer_registry import registry


In [2]:
# Dimensioni ipotetiche
q_dim=512
k_dim=512
v_dim=512
seqlen=1024
batch_size=32
n_heads=8
bias=False
assert q_dim%n_heads==0
head_size=q_dim//n_heads


In [5]:
# Tensori di prova
#query_input_batched=torch.randn(batch_size,seqlen,q_dim)
query_input_unbatched=torch.randn(batch_size*seqlen,q_dim)
#print(f"Batched: {query_input_batched.shape}, Unbatched: {query_input_unbatched.shape}")


In [6]:
## Proiezioni iniziali (a q_dim o 3*q_dim)
#if not is_cross_attention or self.qkv_samedim:
#self.packed_proj = nn.Linear(self.q_dim, 3 * q_dim, bias=bias)
packed_proj=registry.create('linear','linear',in_features=q_dim, out_features=3*q_dim,bias=bias)
#else:
q_proj=registry.create('linear','linear',in_features=q_dim, out_features=q_dim,bias=bias)
#k_proj=registry.create('linear','linear',in_features=k_dim, out_features=q_dim,bias=bias)
#v_proj=registry.create('linear','linear',in_features=v_dim, out_features=q_dim,bias=bias)
#self.q_proj = nn.Linear(q_dim, q_dim, bias=bias)
#self.k_proj = nn.Linear(k_dim, q_dim, bias=bias)
#self.v_proj = nn.Linear(v_dim, q_dim, bias=bias)

In [8]:
#qkv_projected_batched=NormalTensor(tensor=packed_proj(query_input_batched),tensor_order='?S(3*D)')
qkv_projected=NormalTensor(tensor=packed_proj(query_input_unbatched),tensor_order='?S(3*D)')
#print(f"(QKV PACKED PROJECTION): Batched: {qkv_projected_batched.shape}, Unbatched: {qkv_projected_unbatched.shape}")
#qkv_projected_shape='?S(3*D)'

q=NormalTensor(tensor=q_proj(query_input_unbatched),tensor_order='?SD')
#q_unbatched=q_proj(query_input_unbatched)
#print(f"(Q NORMAL PROJECTION): Batched: {q_batched.shape}, Unbatched: {q_unbatched.shape}")
#q_projected_shape='?SD'

In [9]:
q.tensor_order

'?SD'

In [15]:
qkv_projected_heads=NormalTensor(tensor=qkv_projected.tensor.unflatten(-1,[3,n_heads,head_size]),tensor_order='?S3HD')
#qkv_projected_heads_unbatched=qkv_projected_unbatched.unflatten(-1,[3,n_heads,head_size])
q_projected_heads=NormalTensor(tensor=q.tensor.unflatten(-1,[n_heads,head_size]),tensor_order='?SHD')
#q_projected_heads_unbatched=q_unbatched.unflatten(-1,[n_heads,head_size])
#print(f"Packed: batched: {qkv_projected_heads_batched.shape} unbatched: {qkv_projected_heads_unbatched.shape}")
#print(f"Normal: batched: {q_projected_heads_batched.shape} unbatched: {q_projected_heads_unbatched.shape}")


In [39]:
from dataclasses import replace

def _pack_qkv(q, k, v):
    assert q.tensor_order == k.tensor_order == v.tensor_order, "QKV must have same tensor order"
    normalize = lambda s: s.translate(str.maketrans('', '', '?B'))
    current_order_norm = normalize(q.tensor_order)
    
    if current_order_norm == "HSD":
        # [?, H, S, D] => [?, S, H, D]
        q_t = q.tensor.transpose(1, 2)
        k_t = k.tensor.transpose(1, 2)
        v_t = v.tensor.transpose(1, 2)
        #  [?, S, 3, H, D]
        packed_tensor = torch.stack([q_t, k_t, v_t], dim=2)
        new_order = '?S3HD'
    elif current_order_norm == "SHD":
        # [?, S, 3, H, D]
        packed_tensor = torch.stack([q.tensor, k.tensor, v.tensor], dim=2)
        new_order = '?S3HD'
    else:
        raise ValueError(f"Unsupported tensor order: {q.tensor_order}")
    
    return replace(q, tensor=packed_tensor, tensor_order=new_order)

def _unpack_qkv(qkv_packed):
        # S3HD => ?SHD
        q_t, k_t, v_t = qkv_packed.tensor.unbind(dim=1)
        order = qkv_packed.tensor_order.replace('3', '')
        return (replace(qkv_packed, tensor=q_t, tensor_order=order),
                replace(qkv_packed, tensor=k_t, tensor_order=order),
                replace(qkv_packed, tensor=v_t, tensor_order=order))


In [16]:
q_projected_heads.shape

torch.Size([32768, 8, 64])

In [28]:
q_repacked=_pack_qkv(q,q,q)

In [42]:
q.tensor_order

'?SD'

In [35]:
a,b,c=_unpack_qkv(qkv_projected_heads)

In [43]:
d=_pack_qkv(q_projected_heads,q_projected_heads,q_projected_heads)

In [45]:
d.tensor_order

'?S3HD'

In [ ]:
qkv

In [46]:
qkv_projected_heads.tensor_order

'?S3HD'

In [ ]:

def transpose_for_kernel(a,b,identifier):
    print(f"Originale: {identifier}")
    identifier=list(identifier)
    c=identifier[a]
    identifier[a]=identifier[b]
    identifier[b]=c
    return "".join(identifier)
    
    

In [ ]:
transpose(1,2,q_heads_shape)

In [ ]:
q_projected_heads_batched.shape

In [ ]:
prova=q_projected_heads_batched.transpose(1,2)
q_projected_heads_batched.shape
prova.shape

In [ ]:
# Heads creation
            t = qkv_projected.tensor.unflatten(-1, [3, self.nheads, self.head_dim])
            # Detect whether batch dim exists (padded / normal) or not (unpadded represented as (S, 3, H, Hd))
            if t.dim() == 5:
                order = 'S3HD'
            elif t.dim() == 4:
                order = 'S3HD'
            else:
                raise NotImplementedError(f"Unsupported packed qkv tensor shape: {tuple(t.shape)}")
            qkv_projected = NormalTensor(tensor=t, extra_attributes={'tensor_order': order})
        else:
            q = NormalTensor(tensor=q.tensor.unflatten(-1, [self.nheads, self.head_dim]).transpose(1, 2), extra_attributes={'tensor_order': 'HSD'})

In [13]:
'?SHD'.replace('?','')


'SHD'

In [12]:
'BSHD'.replace('B','')

'SHD'

In [16]:
a='BSHD'
a.replace('B','').replace('?','')

'SHD'

In [19]:
q_projected_heads_batched.shape #?SHD

torch.Size([32, 1024, 8, 64])

In [25]:
# Come ripaccare le tre rappresentazioni separate?
repacked=torch.cat([q_projected_heads_batched, q_projected_heads_batched, q_projected_heads_batched], dim=-1)

In [24]:
repacked.shape

torch.Size([32, 1024, 24, 64])